In [21]:
# --- LIBRERÍAS NECESARIAS ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

# --- CONFIGURACIÓN DE SELENIUM ---
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
# options.add_argument("--headless") # Descomenta para que el navegador corra en segundo plano
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)
enlaces = []
datos_trabajos = []


try:
    # --- ETAPA 1: OBTENER LOS ENLACES ---
    print("🚀 INICIANDO ETAPA 1: Obteniendo los enlaces de las ofertas...")
    url_con_filtros = "https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html"
    driver.get(url_con_filtros)
    time.sleep(5)
    print("✅ Página de resultados con filtros cargada.")

    ofertas_raw = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))
    enlaces_temp = [oferta.get_attribute("href") for oferta in ofertas_raw if oferta.get_attribute("href")]
    enlaces = list(set([link for link in enlaces_temp if link and '/empleos/' in link and not '/postulante/' in link]))
    print(f"\n👍 ETAPA 1 COMPLETADA: Se encontraron {len(enlaces)} ofertas de trabajo únicas.")

    # --- ETAPA 2: LÓGICA DE EXTRACCIÓN MULTI-INTENTO ---
    print("\n🚀 INICIANDO ETAPA 2: Extrayendo detalles de cada oferta...")
    for i, link in enumerate(enlaces, start=1):
        try:
            driver.get(link)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # --- ESTRATEGIA DE EXTRACCIÓN ROBUSTA ---
            
            # 1. TÍTULO: Siempre es el <h1>.
            titulo = soup.find('h1').get_text(strip=True) if soup.find('h1') else "No encontrado"
            
            # 2. DESCRIPCIÓN: Se intentan varias formas.
            descripcion = "No encontrada"
            desc_header = soup.find('h3', string='Descripción del puesto')
            if desc_header:
                desc_p = desc_header.find_parent('div').find_next_sibling('p')
                if desc_p: descripcion = desc_p.get_text(strip=True, separator='\\n')
            if descripcion == "No encontrada":
                desc_div = soup.find('p', class_='sc-hqCxVl')
                if desc_div: descripcion = desc_div.get_text(strip=True, separator='\\n')

            # 3. DISTRITO: Se intentan varias formas.
            distrito = "No encontrado"
            distrito_tag = soup.find('h2', class_='sc-gbKBEV')
            if distrito_tag: distrito = distrito_tag.get_text(strip=True)
            if distrito == "No encontrado":
                all_h2 = soup.find_all('h2')
                for h2 in all_h2:
                    if "Peru" in h2.get_text():
                        distrito = h2.get_text(strip=True)
                        break
            
            # 4. MODALIDAD: Búsqueda global de palabras clave.
            modalidad = "No especificada"
            page_text = soup.body.get_text()
            if "Híbrido" in page_text: modalidad = "Híbrido"
            elif "Presencial" in page_text: modalidad = "Presencial"
            elif "Home Office" in page_text or "Remoto" in page_text: modalidad = "Remoto"

            datos_trabajos.append({
                "Job Title": titulo,
                "Description": descripcion,
                "District": distrito,
                "Work Mode": modalidad
            })
            print(f"  -> ✅ Extraída oferta {i}/{len(enlaces)}: {titulo}")
        except Exception as e:
            print(f"  -> ❌ Error en la oferta {i} ({link}) - {type(e).__name__}: {e}")
            
    print("\n👍 ETAPA 2 COMPLETADA: Se extrajeron los detalles.")

finally:
    print("\n🔚 Cerrando navegador...")
    driver.quit()

# --- ETAPA 3: GUARDAR LOS DATOS EN CSV ---
if datos_trabajos:
    print("\n🚀 INICIANDO ETAPA 3: Guardando datos en archivo CSV...")
    df = pd.DataFrame(datos_trabajos)
    output_dir = 'output'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    ruta_salida = os.path.join(output_dir, 'bumeran_jobs.csv')
    df.to_csv(ruta_salida, index=False, encoding='utf-8')
    print(f"\n🎉 ¡TAREA COMPLETADA! Archivo guardado en: {ruta_salida}")
    display(df)
else:
    print("\n⚠️ No se encontraron datos para guardar en el archivo CSV.")


    

🚀 INICIANDO ETAPA 1: Obteniendo los enlaces de las ofertas...
✅ Página de resultados con filtros cargada.

👍 ETAPA 1 COMPLETADA: Se encontraron 20 ofertas de trabajo únicas.

🚀 INICIANDO ETAPA 2: Extrayendo detalles de cada oferta...
  -> ✅ Extraída oferta 1/20: Analista Programador Postgre/Oracle/Híbrido
  -> ✅ Extraída oferta 2/20: Analista Programador - Sede ATE
  -> ✅ Extraída oferta 3/20: Desarrollador Full Stack / Surco / Presencial
  -> ✅ Extraída oferta 4/20: Analista Programador Java
  -> ✅ Extraída oferta 5/20: Desarrollador Backend Semi-Senior
  -> ✅ Extraída oferta 6/20: Desarrollador/ Líder Técnico
  -> ✅ Extraída oferta 7/20: Desarrollador Full Stack Node.js, C#, Java, AngularJS
  -> ✅ Extraída oferta 8/20: Analista de Imagenes Hospitalarias / Sistemas
  -> ✅ Extraída oferta 9/20: Senior React Developer
  -> ✅ Extraída oferta 10/20: Java Developer - Senior
  -> ✅ Extraída oferta 11/20: Trainee Programador Jr
  -> ✅ Extraída oferta 12/20: Programador Full Stack (Java, Reac

,Job Title,Description,District,Work Mode
0,Analista Programador Postgre/Oracle/Híbrido,"Por encargo de nuestro cliente, importante emp...","San Borja, Lima, Peru",Híbrido
1,Analista Programador - Sede ATE,¡Buscamos a nuestro próximo/a Analista Program...,"Ate, Lima, Peru",Presencial
2,Desarrollador Full Stack / Surco / Presencial,"En\nWell Human Resources\n, estamos en la búsq...","Santiago de Surco, Lima, Peru",Híbrido
3,Analista Programador Java,Analista Programador Java\nResumen del Puesto\...,"Lima, Lima, Peru",Híbrido
4,Desarrollador Backend Semi-Senior,Métrica Perú es la primera filial latinoameric...,"Lima, Lima, Peru",Híbrido
5,Desarrollador/ Líder Técnico,En Protiviti Perú buscamos un\nDesarrollador d...,"La Victoria, Lima, Peru",Híbrido
6,"Desarrollador Full Stack Node.js, C#, Java, An...",Puesto: Desarrollador Full Stack Senior\n📍 Mod...,"Lima, Lima, Peru",Híbrido
7,Analista de Imagenes Hospitalarias / Sistemas,Grupo San Pablo te invita a unirte a su equipo...,"Santiago de Surco, Lima, Peru",Híbrido
8,Senior React Developer,Información Importante\nUbicación del cliente:...,Publicado ayer,Híbrido
9,Java Developer - Senior,Actualmente nos encontramos en búsqueda de 02 ...,"Chorrillos, Lima, Peru",Híbrido
